In [1]:
%matplotlib inline

import gensim
import json
import logging
import math
import os
import random
import re
import spacy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyLDAvis.gensim_models
import warnings

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.test.utils import datapath

from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import TrigramCollocationFinder
from nltk.metrics import TrigramAssocMeasures
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.preprocessing import normalize

from stop_words import get_stop_words
from wordcloud import WordCloud

In [2]:
with open(os.path.join('posts_maior_propre.json'), 'r', encoding = 'UTF-8') as fin_M2 :
    data = json.load(fin_M2)

In [3]:
index = ''
a = ''
list_clean_content = []

for j in range(len(data)) :
    index = data[j]
    a = index.replace( "\n", "")
    list_clean_content.append(a)

In [4]:
def top_feats(row, features, top_n):
    top_n_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in top_n_ids if row[i]>0]
    df = pd.DataFrame(top_feats)
    if len(top_feats) > 0:
        df.columns = [ 'feature', 'score']
    return df

def cosinus(i, j):
    num = np.dot(i, j)
    den = math.sqrt(sum(i*i))*math.sqrt(sum(j*j))
    if (den>0):    
        return (num/den)
    else:
        return 0
    
def search(q, D):
    cc = {i: cosinus(D[i, :], q) for i in range(n_docs)}
    cc = sorted(cc.items(), key = lambda x: x[1], reverse = True)
    return cc

In [5]:
lines = list_clean_content

In [6]:
sp_stop_words = get_stop_words('spanish')
fr_stop_words = get_stop_words('french')
en_stop_words = get_stop_words('english')

words = [ 'plus', 'rien', 'tres', 'ca', 'fr', 'com', 'www', 'http',
          'etais', 'etait', 'etaient', 'meme', 'non', 'bien', 'oui', 'quoi',
          'chez', 'va', 'apres']

stopwords = sp_stop_words + fr_stop_words + en_stop_words + words 

In [7]:
pd.set_option('display.max_rows', None)
Y_tfidf_vect = TfidfVectorizer( stop_words = stopwords, max_features = 5000).fit(lines)
Y_feature_names = Y_tfidf_vect.get_feature_names()
Y_matrix_TFxIDF = Y_tfidf_vect.transform(lines)
A = Y_matrix_TFxIDF.toarray()
n_docs, n_terms = A.shape
tf_sum = np.sum( A, axis = 0)
top_feats( tf_sum, Y_feature_names, 500)

C:\Users\filip\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(
C:\Users\filip\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,feature,score
0,jour,1653.073928
1,temps,1635.419369
2,etc,1523.309633
3,vie,1395.621426
4,monde,1374.329444
5,nuit,1229.072934
6,après,1197.262350
7,dit,1177.747515
8,autres,1132.858813
9,toujours,1102.759488


In [34]:
query = [ 'signez', 'petition'] 
indexes = [Y_feature_names.index(q) for q in query if q in Y_feature_names]
print(indexes)

[]


In [35]:
query_vec = np.zeros(n_terms)
query_vec[indexes] = 1
result = search( query_vec, A)
result[0:10]

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0),
 (9, 0)]

In [36]:
nb_top_docs = 10
top_docs = [r for (r,v) in result[0:nb_top_docs]]
print(top_docs)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [37]:
for i, td in zip(range(nb_top_docs), top_docs):
    print("%s (%s): %s" % (i+1, td, lines[td]))

1 (0): le premier jour du premier mois de l'année ressemble étrangement au dernier jour du dernier mois de l'année plus rien ne distingue l'aube du crépuscule et toi qui flottes à mes côtés lorsque que tu me regardes ainsi j'ai l'impression d'être sauvé il neige des noms et jamais je n'en retiens un seulsous ce ciel là la terre qui rêve et qui tourne semble soudain souterraine bornes de bitume dans les odeurs saines de l'hiver j'enchaîne des dizaines de patelins l'état a un employeur qui  créé des poisons avant que de vendre ses antidotes et toi te l'ai-je déjà dit à peine me touches-tu que mes cicatrices disparaissent il neige des noms et jamais je n'en retiens un seul splendeur sérénité sandwich thon crudités sainte trinité qui me rend immortel sur l'aire de repos tel le poète maudit de baseje porte un bonnet post apo et comme n'importe quel ado déséquilibré je me suis offert un couteau de pocheet toi que je cherche depuis tant de temps je m'aperçois qu'en fait tu as toujours été là 

In [12]:
texte = str(list_clean_content)

In [13]:
type(texte)

str

In [14]:
len(texte)

166328434

In [15]:
texte = texte[:400000]

In [16]:
words = [w.lower() for w in word_tokenize(texte)]

In [17]:
bcf = BigramCollocationFinder.from_words(words)
bcf.nbest(BigramAssocMeasures.likelihood_ratio, 10)
stopset = stopwords
filter_stops = lambda w: len(w) < 3 or w in stopset
bcf.apply_word_filter(filter_stops)
bcf.nbest(BigramAssocMeasures.likelihood_ratio, 10)

[('fabrice', 'farre'),
 ('heptanes', 'fraxion'),
 ('quelque', 'chose'),
 ('version', 'originale'),
 ('encres', 'vives'),
 ('tel', '+33'),
 ('sophie', 'brassart'),
 ("qu'il", 'faut'),
 ('amer', 'rhésus'),
 ('travail', 'idiotvivre')]

In [18]:
tcf = TrigramCollocationFinder.from_words(words)
tcf.apply_word_filter(filter_stops)
tcf.apply_freq_filter(3)
tcf.nbest(TrigramAssocMeasures.likelihood_ratio, 10)

[("qu'il", 'faut', 'absolument'),
 ('amer', 'rhésus', 'amer'),
 ('amer', 'amer', 'rhésus'),
 ("s'appellent", 'xautant', "d'âmes"),
 ('filles', "s'appellent", 'xautant'),
 ('bruno', 'guattari', 'éditeur'),
 ('fête', 'méchante', 'fête'),
 ('patty', 'dickson', 'pieczka'),
 ('pièce', 'manquante', "c'est"),
 ('calme', 'chiencalme', 'chienje')]

In [19]:
best_big = bcf.nbest(BigramAssocMeasures.raw_freq, 10)

In [20]:
Y_tfidf_vect = TfidfVectorizer( stop_words = stopwords, ngram_range=(2, 2),max_features = 5000).fit(lines)
Y_feature_names = Y_tfidf_vect.get_feature_names()
Y_matrix_TFxIDF = Y_tfidf_vect.transform(lines)
A = Y_matrix_TFxIDF.toarray()
n_docs, n_terms = A.shape
tf_sum = np.sum( A, axis = 0)
top_feats( tf_sum, Y_feature_names, 500)

C:\Users\filip\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(
C:\Users\filip\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,feature,score
0,aujourd hui,1802.579677
1,quelque chose,1375.738338
2,aimé post,894.103755
3,post apprécierez,893.184876
4,apprécierez celui,892.559073
5,après midi,636.346780
6,etc etc,627.433832
7,quelque part,448.827680
8,rights reserved,360.989930
9,patricia laranco,355.549553


In [21]:
Y_tfidf_vect = TfidfVectorizer( stop_words = stopwords, ngram_range=(3, 3),max_features = 5000).fit(lines)
Y_feature_names = Y_tfidf_vect.get_feature_names()
Y_matrix_TFxIDF = Y_tfidf_vect.transform(lines)
A = Y_matrix_TFxIDF.toarray()
n_docs, n_terms = A.shape
tf_sum = np.sum( A, axis = 0)
top_feats( tf_sum, Y_feature_names, 500)

C:\Users\filip\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(
C:\Users\filip\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,feature,score
0,aimé post apprécierez,1438.615086
1,post apprécierez celui,1435.668971
2,etc etc etc,617.820799
3,fin après midi,225.242509
4,wikipedia org wiki,222.065700
5,act colorofchange org,157.514824
6,colorofchange org go,156.738849
7,jour après jour,150.519370
8,laranco rights reserved,122.840181
9,photographies patricia laranco,121.874301
